# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集

訓練一個 CNN 模型來學習門牌號碼資料集。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1260
┌ Warning: Module MbedTLS with build ID 4906264239951 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1016
┌ Warning: Module MbedTLS with build ID 4906264239951 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1016
┌ Warning: Module MbedTLS with build ID 4906264239951 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1016
┌ Warning: Module MbedTLS with build ID 4906264239951 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is i

In [2]:
import CuArrays
CuArrays.allowscalar(false)

## 讀取資料

In [3]:
train_X, train_y = SVHN2.traindata(Float32, 1:20000)
test_X,  test_y  = SVHN2.testdata(Float32, 1:2000)

(Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627451 … 0.647

In [4]:
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

10×2000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  1  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  1  0  0  0  0  0  1  0  1  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [5]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test = DataLoader(test_X, test_y, batchsize=batchsize)

DataLoader((Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627

In [6]:
train = gpu(train)
test = gpu(test)
println(size(train_X))

(32, 32, 3, 20000)


## CNN 模型

In [7]:
model = Chain(
    Conv((3,3), 3 => 16, relu),
    MaxPool((2,2)),
    BatchNorm(16, relu),
    Conv((3,3), 16 => 32, relu),
    MaxPool((2,2)),
    BatchNorm(32, relu),
    Conv((3,3), 32 => 64, relu),
    MaxPool((2,2)),
    BatchNorm(64, relu),
    flatten,
    Dense(256, 10)
)# write your model here

Chain(Conv((3, 3), 3=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(16, λ = relu), Conv((3, 3), 16=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(32, λ = relu), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(64, λ = relu), flatten, Dense(256, 10))

In [8]:
model = gpu(model)

Chain(Conv((3, 3), 3=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(16, λ = relu), Conv((3, 3), 16=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(32, λ = relu), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(64, λ = relu), flatten, Dense(256, 10))

In [9]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

In [10]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end
evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

In [14]:
epochs = 5
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main /home/ghost/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


test_loss() = 1.594378f0
test_loss() = 0.7555119f0
test_loss() = 0.8143116f0
test_loss() = 0.66146636f0


┌ Info: Epoch 2
└ @ Main /home/ghost/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


test_loss() = 1.7016221f0
test_loss() = 0.73580635f0
test_loss() = 0.79853165f0


┌ Info: Epoch 3
└ @ Main /home/ghost/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


test_loss() = 1.04096f0
test_loss() = 0.6482898f0
test_loss() = 0.6384344f0


┌ Info: Epoch 4
└ @ Main /home/ghost/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


test_loss() = 1.1812334f0
test_loss() = 0.7606853f0
test_loss() = 0.6896684f0


┌ Info: Epoch 5
└ @ Main /home/ghost/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


test_loss() = 1.303179f0
test_loss() = 0.7255899f0
test_loss() = 0.5714537f0


In [18]:
function accuracy(data_loader, model)
    acc = 0
    for (x,y) in data_loader
        
        acc += sum(onecold(cpu(model(x))) .== onecold(cpu(y)))*1 / size(x,4)
    end
    acc/length(data_loader)
end

accuracy (generic function with 1 method)

In [19]:
accuracy(train, model)

0.8895450367647058

In [20]:
accuracy(test, model)

0.8103467597336065